# 03 - Feature Engineering

**Objective**: Create the target variable, engineer features, encode categoricals, scale numerics, and prepare train/test splits ready for model training.

**Input**: `data/processed/cleaned_dataset.csv`  
**Output**: `data/processed/featured_dataset.csv` + PySpark prepared train/test DataFrames saved as parquet

In [ ]:
# ============================================================
# CELL 1: Imports & Spark Session
# ============================================================

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

spark = SparkSession.builder \
    .appName("FeatureEngineering") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

DATA_DIR = r'F:\SOFTWARICA\big-data-transport-analytics\data\processed'
MODEL_DIR = os.path.join(DATA_DIR, 'models')
OUTPUT_DIR = r'F:\SOFTWARICA\big-data-transport-analytics\outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print("Spark session ready!")

Spark session ready!


In [2]:
# ============================================================
# CELL 2: Load Cleaned Dataset
# ============================================================

pdf = pd.read_csv(os.path.join(DATA_DIR, 'cleaned_dataset.csv'))
print(f"Loaded cleaned dataset: {pdf.shape[0]:,} rows x {pdf.shape[1]} columns")
print(f"\nColumns: {list(pdf.columns)}")
pdf.head(3)

Loaded cleaned dataset: 34,922 rows x 16 columns

Columns: ['line_name', 'direction', 'stop_sequence', 'longitude', 'latitude', 'run_time_min', 'departure_hour', 'is_peak_hour', 'day_of_week_num', 'active_disruptions', 'unique_situations', 'avg_severity_score', 'planned_count', 'unplanned_count', 'roadworks_count', 'roadclosed_count']


,line_name,direction,stop_sequence,longitude,latitude,run_time_min,departure_hour,is_peak_hour,day_of_week_num,active_disruptions,unique_situations,avg_severity_score,planned_count,unplanned_count,roadworks_count,roadclosed_count
0,22,outbound,1,0.287746,51.485761,1.0,17.0,1,0,82,68,1.609756,60,22,43,16
1,22,outbound,2,0.288737,51.487744,1.0,17.0,1,0,82,68,1.609756,60,22,43,16
2,22,outbound,3,0.282997,51.490458,4.0,17.0,1,0,82,68,1.609756,60,22,43,16


In [3]:
# ============================================================
# CELL 3: Create Target Variable & Engineer Features
# ============================================================

print("=" * 60)
print("TARGET VARIABLE: high_disruption_risk")
print("=" * 60)

# Understand disruption distribution
print("\nactive_disruptions distribution:")
print(pdf['active_disruptions'].describe())

print(f"\nPercentiles:")
for p in [25, 50, 60, 70, 75, 80, 90]:
    val = pdf['active_disruptions'].quantile(p / 100)
    print(f"  {p}th percentile: {val:.0f}")

# Use MEDIAN as threshold -> balanced classes
THRESHOLD = pdf['active_disruptions'].median()
print(f"\n>>> Chosen threshold: {THRESHOLD:.0f} (median)")

# Create binary target
pdf['high_disruption_risk'] = (pdf['active_disruptions'] > THRESHOLD).astype(int)

print(f"\nTarget distribution:")
print(pdf['high_disruption_risk'].value_counts())
print(f"\nClass balance: {pdf['high_disruption_risk'].mean():.2%} positive (high risk)")

# ---- Feature Engineering (timetable-derived only) ----
print("\n" + "=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)

# 1. Weekend flag from day_of_week_num (5=Sat, 6=Sun)
pdf['is_weekend'] = (pdf['day_of_week_num'] >= 5).astype(int)
print(f"  + is_weekend (Sat/Sun flag)")

# 2. Cyclical encoding of departure_hour (captures 23→0 wraparound)
pdf['hour_sin'] = np.sin(2 * np.pi * pdf['departure_hour'] / 24)
pdf['hour_cos'] = np.cos(2 * np.pi * pdf['departure_hour'] / 24)
print(f"  + hour_sin, hour_cos (cyclical hour encoding)")

# 3. Time-of-day bins (morning rush, midday, evening rush, night)
bins = [0, 6, 10, 16, 20, 24]
labels_td = ['night', 'morning_rush', 'midday', 'evening_rush', 'late_night']
pdf['time_of_day'] = pd.cut(pdf['departure_hour'], bins=bins, labels=labels_td, right=False)
pdf['time_of_day'] = pdf['time_of_day'].astype(str)
print(f"  + time_of_day (5 bins: {labels_td})")

# 4. Geographic zone (cluster stops by latitude bands)
pdf['lat_zone'] = pd.qcut(pdf['latitude'], q=4, labels=['south', 'mid_south', 'mid_north', 'north'])
pdf['lat_zone'] = pdf['lat_zone'].astype(str)
print(f"  + lat_zone (4 latitude quartile zones)")

# 5. Route complexity proxy (stops × run_time interaction)
pdf['route_complexity'] = pdf['stop_sequence'] * pdf['run_time_min']
print(f"  + route_complexity (stop_sequence × run_time_min)")

# Drop ALL disruption-derived columns — they leak the target
LEAKAGE_COLS = [
    'active_disruptions',
    'unique_situations',
    'avg_severity_score',
    'planned_count',
    'unplanned_count',
    'roadworks_count',
    'roadclosed_count',
]
pdf = pdf.drop(columns=LEAKAGE_COLS)
print(f"\nDropped {len(LEAKAGE_COLS)} disruption columns (ALL leak target)")
print(f"Dataset now: {pdf.shape[0]:,} rows x {pdf.shape[1]} columns")
print(f"Columns: {list(pdf.columns)}")

TARGET VARIABLE: high_disruption_risk

active_disruptions distribution:
count    34922.000000
mean       111.791421
std         40.022408
min          3.000000
25%         84.000000
50%        107.000000
75%        107.000000
max        189.000000
Name: active_disruptions, dtype: float64

Percentiles:
  25th percentile: 84
  50th percentile: 107
  60th percentile: 107
  70th percentile: 107
  75th percentile: 107
  80th percentile: 173
  90th percentile: 173

>>> Chosen threshold: 107 (median)

Target distribution:
high_disruption_risk
0    26774
1     8148
Name: count, dtype: int64

Class balance: 23.33% positive (high risk)

FEATURE ENGINEERING
  + is_weekend (Sat/Sun flag)
  + hour_sin, hour_cos (cyclical hour encoding)
  + time_of_day (5 bins: ['night', 'morning_rush', 'midday', 'evening_rush', 'late_night'])
  + lat_zone (4 latitude quartile zones)
  + route_complexity (stop_sequence × run_time_min)

Dropped 7 disruption columns (ALL leak target)
Dataset now: 34,922 rows x 16 colu

In [5]:
# ============================================================
# CELL 4: EDA - Exploratory Visualizations
# ============================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Exploratory Data Analysis - Timetable Features vs Risk', fontsize=14, fontweight='bold')

# 1. Target distribution
pdf['high_disruption_risk'].value_counts().plot(
    kind='bar', ax=axes[0, 0], color=['#2ecc71', '#e74c3c'])
axes[0, 0].set_title('Target: High Disruption Risk')
axes[0, 0].set_xticklabels(['Low Risk (0)', 'High Risk (1)'], rotation=0)
axes[0, 0].set_ylabel('Count')

# 2. Departure hour by risk
for risk, color in [(0, '#2ecc71'), (1, '#e74c3c')]:
    subset = pdf[pdf['high_disruption_risk'] == risk]
    axes[0, 1].hist(subset['departure_hour'], bins=20, alpha=0.6,
                    label=f'Risk={risk}', color=color)
axes[0, 1].set_title('Departure Hour by Risk Level')
axes[0, 1].set_xlabel('Hour of Day')
axes[0, 1].legend()

# 3. Run time by risk
pdf.boxplot(column='run_time_min', by='high_disruption_risk', ax=axes[0, 2])
axes[0, 2].set_title('Run Time by Risk')
axes[0, 2].set_xlabel('High Disruption Risk')
plt.sca(axes[0, 2]); plt.title('Run Time (minutes) by Risk')

# 4. Risk by time of day
time_risk = pdf.groupby(['time_of_day', 'high_disruption_risk']).size().unstack(fill_value=0)
time_risk.plot(kind='bar', ax=axes[1, 0], color=['#2ecc71', '#e74c3c'])
axes[1, 0].set_title('Risk by Time of Day')
axes[1, 0].set_xlabel('Time Period')
axes[1, 0].legend(['Low Risk', 'High Risk'])
axes[1, 0].tick_params(axis='x', rotation=45)

# 5. Risk by latitude zone
lat_risk = pdf.groupby(['lat_zone', 'high_disruption_risk']).size().unstack(fill_value=0)
lat_risk.plot(kind='bar', ax=axes[1, 1], color=['#2ecc71', '#e74c3c'])
axes[1, 1].set_title('Risk by Geographic Zone')
axes[1, 1].set_xlabel('Latitude Zone')
axes[1, 1].legend(['Low Risk', 'High Risk'])
axes[1, 1].tick_params(axis='x', rotation=45)

# 6. Route complexity by risk
pdf.boxplot(column='route_complexity', by='high_disruption_risk', ax=axes[1, 2])
axes[1, 2].set_title('Route Complexity by Risk')
plt.sca(axes[1, 2]); plt.title('Route Complexity by Risk')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'eda_plots.png'), dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/eda_plots.png")

Saved: outputs/eda_plots.png


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_31372\2007104469.py:53: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
# ============================================================
# CELL 5: Correlation Heatmap
# ============================================================

numeric_cols = pdf.select_dtypes(include=[np.number]).columns.tolist()

fig, ax = plt.subplots(figsize=(12, 9))
corr = pdf[numeric_cols].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdYlBu_r', center=0,
            square=True, ax=ax, linewidths=0.5)
ax.set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'correlation_heatmap.png'), dpi=150, bbox_inches='tight')
plt.show()

# Correlations with target
print("Correlation with target (high_disruption_risk):")
target_corr = corr['high_disruption_risk'].drop('high_disruption_risk').sort_values(ascending=False)
for name, val in target_corr.items():
    if pd.isna(val):
        print(f"  {name:25s}  NaN (constant or missing)")
        continue
    bar = '█' * int(abs(val) * 50)
    sign = '+' if val > 0 else '-'
    print(f"  {name:25s} {sign}{abs(val):.4f} {bar}")

print("\nSaved: outputs/correlation_heatmap.png")

Correlation with target (high_disruption_risk):
  is_weekend                +0.5159 █████████████████████████
  run_time_min              +0.0422 ██
  is_peak_hour              +0.0161 
  hour_sin                  -0.0049 
  departure_hour            -0.0096 
  longitude                 -0.0166 
  day_of_week_num           -0.0219 █
  latitude                  -0.0348 █
  route_complexity          -0.0390 █
  hour_cos                  -0.0397 █
  stop_sequence             -0.1074 █████

Saved: outputs/correlation_heatmap.png


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_31372\3451154644.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
# ============================================================
# CELL 6: Define Feature & Target Columns
# ============================================================

# Categorical features (need encoding → one-hot)
CAT_FEATURES = ['line_name', 'direction', 'time_of_day', 'lat_zone']

# Numeric features (timetable + engineered)
NUM_FEATURES = [
    'departure_hour',
    'is_peak_hour',
    'day_of_week_num',
    'stop_sequence',
    'latitude',
    'longitude',
    'run_time_min',
    'is_weekend',
    'hour_sin',
    'hour_cos',
    'route_complexity',
]

TARGET = 'high_disruption_risk'

ALL_FEATURES = CAT_FEATURES + NUM_FEATURES

print("FEATURE SUMMARY")
print("=" * 50)
print(f"Categorical features ({len(CAT_FEATURES)}):")
for col in CAT_FEATURES:
    print(f"  - {col:20s} ({pdf[col].nunique()} unique values)")
print(f"\nNumeric features ({len(NUM_FEATURES)}):")
for col in NUM_FEATURES:
    print(f"  - {col}")
print(f"\nTarget: {TARGET}")
print(f"Total base features: {len(ALL_FEATURES)}")
print(f"  (expands after one-hot encoding categoricals)")
print(f"\nNOTE: Zero disruption-derived features used — prevents leakage.")

FEATURE SUMMARY
Categorical features (4):
  - line_name            (12 unique values)
  - direction            (3 unique values)
  - time_of_day          (5 unique values)
  - lat_zone             (4 unique values)

Numeric features (11):
  - departure_hour
  - is_peak_hour
  - day_of_week_num
  - stop_sequence
  - latitude
  - longitude
  - run_time_min
  - is_weekend
  - hour_sin
  - hour_cos
  - route_complexity

Target: high_disruption_risk
Total base features: 15
  (expands after one-hot encoding categoricals)

NOTE: Zero disruption-derived features used — prevents leakage.


In [8]:
# ============================================================
# CELL 7: Load into PySpark & Build Preprocessing Pipeline
# ============================================================

# Load pandas DF into Spark
df = spark.createDataFrame(pdf[ALL_FEATURES + [TARGET]])
print(f"Spark DataFrame: {df.count():,} rows")
df.printSchema()

# --- Build Pipeline ---
# Step 1: StringIndexer for each categorical
indexers = [
    StringIndexer(inputCol=col, outputCol=f'{col}_idx', handleInvalid='keep')
    for col in CAT_FEATURES
]

# Step 2: OneHotEncoder for each indexed categorical
encoders = [
    OneHotEncoder(inputCol=f'{col}_idx', outputCol=f'{col}_vec')
    for col in CAT_FEATURES
]

# Step 3: Assemble all features into a single vector
encoded_cols = [f'{col}_vec' for col in CAT_FEATURES]
assembler = VectorAssembler(
    inputCols=NUM_FEATURES + encoded_cols,
    outputCol='features_raw'
)

# Step 4: StandardScaler (important for Logistic Regression)
scaler = StandardScaler(
    inputCol='features_raw', outputCol='features',
    withStd=True, withMean=False
)

preprocessing_pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler])

print("\nPreprocessing pipeline:")
for i, stage in enumerate(preprocessing_pipeline.getStages(), 1):
    print(f"  {i}. {type(stage).__name__}")

Spark DataFrame: 34,922 rows
root
 |-- line_name: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- lat_zone: string (nullable = true)
 |-- departure_hour: double (nullable = true)
 |-- is_peak_hour: long (nullable = true)
 |-- day_of_week_num: long (nullable = true)
 |-- stop_sequence: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- run_time_min: double (nullable = true)
 |-- is_weekend: long (nullable = true)
 |-- hour_sin: double (nullable = true)
 |-- hour_cos: double (nullable = true)
 |-- route_complexity: double (nullable = true)
 |-- high_disruption_risk: long (nullable = true)


Preprocessing pipeline:
  1. StringIndexer
  2. StringIndexer
  3. StringIndexer
  4. StringIndexer
  5. OneHotEncoder
  6. OneHotEncoder
  7. OneHotEncoder
  8. OneHotEncoder
  9. VectorAssembler
  10. StandardScaler


In [9]:
# ============================================================
# CELL 8: Train/Test Split (80/20)
# ============================================================

# Split BEFORE fitting pipeline (prevents data leakage)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

print(f"Train set: {train_df.count():,} rows ({train_df.count()/df.count()*100:.1f}%)")
print(f"Test set:  {test_df.count():,} rows ({test_df.count()/df.count()*100:.1f}%)")

print(f"\nTrain class distribution:")
train_df.groupBy(TARGET).count().orderBy(TARGET).show()

print(f"Test class distribution:")
test_df.groupBy(TARGET).count().orderBy(TARGET).show()

Train set: 28,119 rows (80.5%)
Test set:  6,803 rows (19.5%)

Train class distribution:
+--------------------+-----+
|high_disruption_risk|count|
+--------------------+-----+
|                   0|21550|
|                   1| 6569|
+--------------------+-----+

Test class distribution:
+--------------------+-----+
|high_disruption_risk|count|
+--------------------+-----+
|                   0| 5224|
|                   1| 1579|
+--------------------+-----+



In [10]:
# ============================================================
# CELL 9: Fit Pipeline on Train, Transform Both
# ============================================================

# Fit on training data ONLY
pipeline_model = preprocessing_pipeline.fit(train_df)

# Transform both sets
train_prepared = pipeline_model.transform(train_df)
test_prepared = pipeline_model.transform(test_df)

feature_size = train_prepared.select('features').first()[0].size
print(f"Pipeline fitted on training data.")
print(f"Feature vector size: {feature_size}")

# Show encodings for each categorical
n_indexers = len(CAT_FEATURES)
for i, col in enumerate(CAT_FEATURES):
    labels = pipeline_model.stages[i].labels
    print(f"\n{col} encoding ({len(labels)} categories):")
    for j, label in enumerate(labels):
        print(f"  {label} -> {j}")

print(f"\nSample prepared row:")
train_prepared.select('features', TARGET).show(3, truncate=False)

Pipeline fitted on training data.
Feature vector size: 35

line_name encoding (12 categories):
  22 -> 0
  73 -> 1
  33 -> 2
  83 -> 3
  x80 -> 4
  44 -> 5
  99 -> 6
  88 -> 7
  99OT -> 8
  x32 -> 9
  x2 -> 10
  x1 -> 11

direction encoding (3 categories):
  outbound -> 0
  inbound -> 1
  clockwise -> 2

time_of_day encoding (5 categories):
  midday -> 0
  morning_rush -> 1
  evening_rush -> 2
  late_night -> 3
  night -> 4

lat_zone encoding (4 categories):
  mid_north -> 0
  south -> 1
  mid_south -> 2
  north -> 3

Sample prepared row:
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|features                                                                                                                                         

In [ ]:
# ============================================================
# CELL 10: Save Everything for Model Training
# ============================================================

# Save the featured pandas dataset (with target, before Spark encoding)
pdf.to_csv(os.path.join(DATA_DIR, 'featured_dataset.csv'), index=False)
print(f"Saved: data/processed/featured_dataset.csv ({pdf.shape[0]:,} x {pdf.shape[1]})")

# Save train/test splits as CSV (avoids Hadoop/winutils issues on Windows)
train_pdf = train_df.toPandas()
test_pdf = test_df.toPandas()
train_pdf.to_csv(os.path.join(DATA_DIR, 'train_split.csv'), index=False)
test_pdf.to_csv(os.path.join(DATA_DIR, 'test_split.csv'), index=False)
print(f"Saved: data/processed/train_split.csv ({len(train_pdf):,} rows)")
print(f"Saved: data/processed/test_split.csv ({len(test_pdf):,} rows)")

# Save metadata for next notebook — include all categorical label mappings
import json

cat_label_map = {}
n_indexers = len(CAT_FEATURES)
for i, col in enumerate(CAT_FEATURES):
    cat_label_map[col] = list(pipeline_model.stages[i].labels)

metadata = {
    'target': TARGET,
    'threshold': float(THRESHOLD),
    'num_features': NUM_FEATURES,
    'cat_features': CAT_FEATURES,
    'feature_vector_size': feature_size,
    'train_count': len(train_pdf),
    'test_count': len(test_pdf),
    'cat_labels': cat_label_map,
}
with open(os.path.join(MODEL_DIR, 'feature_metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"Saved: data/processed/models/feature_metadata.json")

print(f"\n--- Feature Engineering Complete ---")
print(f"  Ready for 04_model_training.ipynb")

Saved: data/processed/featured_dataset.csv (34,922 x 16)
Saved: data/processed/train_split.csv (28,119 rows)
Saved: data/processed/test_split.csv (6,803 rows)
Saved: data/processed/feature_metadata.json

--- Feature Engineering Complete ---
  Ready for 04_model_training.ipynb
